**Step 1:**

I have got pre-processed fastq files (guaterria and non-guaterria plastomes) assembled to uvaria afzelii with annotations (incl. from dogma). I preprocess the files into fasta format in Python and save them to the same folder as the original. 

For Guatteria: D:\Annonaceae plastome data\Annonaceae plastome data\Plastome data\Guatteria

In [48]:
import glob
from Bio import SeqIO
import os

input_folder_guatteria = r"D:\Annonaceae plastome data\Annonaceae plastome data\Plastome data\Guatteria"

# Find all FASTQ files in that folder
fastq_files_guatteria = glob.glob(os.path.join(input_folder_guatteria, "*.fastq"))

for fastq_file_guatteria in fastq_files_guatteria:
    output_file_guatteria = os.path.splitext(fastq_file_guatteria)[0] + ".fasta"
    with open(output_file_guatteria, "w") as out_handle:
        count = SeqIO.convert(fastq_file_guatteria, "fastq", out_handle, "fasta")
    #print(f"Converted {count} records from {fastq_file} to {output_file}")


For non-Guatteria:

In [49]:
input_folder_non_guatteria = r"D:\Annonaceae plastome data\Annonaceae plastome data\Plastome data\non-Guatteria"

# Find all FASTQ files in that folder
fastq_files_non_guatteria = glob.glob(os.path.join(input_folder_non_guatteria, "*.fastq"))

for fastq_file_non_guatteria in fastq_files_non_guatteria:
    output_file_non_guatteria = os.path.splitext(fastq_file_non_guatteria)[0] + ".fasta"
    with open(output_file_non_guatteria, "w") as out_handle:
        count = SeqIO.convert(fastq_file_non_guatteria, "fastq", out_handle, "fasta")
    #print(f"Converted {count} records from {fastq_file} to {output_file}")

**Step 2:**

Prepare Input Files (FASTA) - may want to combine all FASTA sequences into one file:

In [50]:
# Collect all fasta files from both directories
input_files_guatteria = glob.glob(os.path.join(input_folder_guatteria, "*.fasta"))
input_files_non_guatteria = glob.glob(os.path.join(input_folder_non_guatteria, "*.fasta"))

# Combine the file lists
input_files_mafft = input_files_guatteria + input_files_non_guatteria

# Output file
combined_file_mafft = "all_plastomes.fasta"

# Write all sequences to the combined file
with open(combined_file_mafft, "w") as outfile:
    for fname in input_files_mafft:
        for record in SeqIO.parse(fname, "fasta"):
            SeqIO.write(record, outfile, "fasta")


**Step 3:**

Run MAFFT from Python

In [51]:
import subprocess

input_fasta_mafft = "all_plastomes.fasta"
output_fasta_mafft = "aligned_plastomes.fasta"

# Adjust `--auto` for best general results, or use `--globalpair --maxiterate 1000` for accuracy
mafft_path = r"D:\mafft\mafft-win\mafft.bat"  # Use raw string (r"...")

cmd = [mafft_path, "--auto", input_fasta_mafft]

with open(output_fasta_mafft, "w") as out_f:
    subprocess.run(cmd, stdout=out_f)

print("Alignment completed with MAFFT.")

Alignment completed with MAFFT.


**Step 4:**

Snip the erroneous edges using sliding windows (automated curation)

Manual curation done in AliView (which turned out better that basic sliding windows - *trimmed_alignment.fasta*) and therefore will be used in further processing unter the name *aligned_plastomes_copy_aliview* 

Stricter version of sliding windows (*trimmed_alignment_stricter.fasta*) set up such that no more than 20% gaps are allowed and in a column 80% needs to agree, otherwise discarded

In [52]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment

def trim_alignment(input_file, output_file, gap_threshold=0.5, window_size=20):
    alignment = AlignIO.read(input_file, "fasta")
    aln_len = alignment.get_alignment_length()
    n_seqs = len(alignment)

    def calc_gap_fraction(column):
        gaps = sum(1 for c in column if c in ["-", "N", "n"])
        return gaps / n_seqs

    # Find good alignment positions
    good_pos = []
    for i in range(aln_len):
        col = alignment[:, i]
        if calc_gap_fraction(col) < gap_threshold:
            good_pos.append(i)

    # Find longest continuous stretch of good positions (with window)
    start, end = 0, 0
    current = []
    for i in range(len(good_pos) - window_size + 1):
        window = good_pos[i:i+window_size]
        if window[-1] - window[0] == window_size - 1:
            current = window
            break

    if not current:
        raise ValueError("No clean window found with given parameters.")

    # Define final trim region
    trim_start = current[0]
    trim_end = good_pos[-1]

    # Trim and write output
    trimmed = MultipleSeqAlignment([
        record[trim_start:trim_end+1] for record in alignment
    ])


    AlignIO.write(trimmed, output_file, "fasta")
    print(f"Trimmed alignment written to: {output_file}")
    print(f"Trimmed from columns {trim_start} to {trim_end} ({trim_end - trim_start + 1} bp)")
    print(f"Original columns: {aln_len}, Removed: {aln_len - (trim_end - trim_start + 1)}")

trim_alignment("aligned_plastomes.fasta", "trimmed_alignment.fasta")


Trimmed alignment written to: trimmed_alignment.fasta
Trimmed from columns 1429 to 222861 (221433 bp)
Original columns: 223075, Removed: 1642


The stricter version of the above cell:

In [53]:
from collections import Counter

def trim_alignment_stricter(input_file, output_file, gap_threshold=0.2, consensus_threshold=0.8, window_size=30):
    alignment = AlignIO.read(input_file, "fasta")
    aln_len = alignment.get_alignment_length()
    n_seqs = len(alignment)

    gap_fractions = []
    consensus_scores = []
    good_pos = []

    def is_column_good(column):
        gaps = sum(1 for c in column if c in ["-", "N", "n"])
        gap_frac = gaps / n_seqs
        bases = [c for c in column if c not in ["-", "N", "n"]]
        consensus = 0
        if bases:
            consensus = Counter(bases).most_common(1)[0][1] / len(bases)

        gap_fractions.append(gap_frac)
        consensus_scores.append(consensus)
        return gap_frac <= gap_threshold and consensus >= consensus_threshold

    for i in range(aln_len):
        if is_column_good(alignment[:, i]):
            good_pos.append(i)

    # Find longest clean window
    start, end = 0, 0
    for i in range(len(good_pos) - window_size + 1):
        window = good_pos[i:i + window_size]
        if window[-1] - window[0] == window_size - 1:
            start = window[0]
            break
    else:
        raise ValueError("No clean window found with given parameters.")

    end = good_pos[-1]

    # Trim and write output
    trimmed = MultipleSeqAlignment([
        record[start:end + 1] for record in alignment
    ])
    AlignIO.write(trimmed, output_file, "fasta")

    # Report
    print(f"Trimmed alignment written to: {output_file}")
    print(f"Trimmed from columns {start} to {end} ({end - start + 1} bp)")
    print(f"Original columns: {aln_len}, Removed: {aln_len - (end - start + 1)}")

# Run it
trim_alignment_stricter("aligned_plastomes.fasta", "trimmed_alignment_stricter.fasta")


Trimmed alignment written to: trimmed_alignment_stricter.fasta
Trimmed from columns 3421 to 222728 (219308 bp)
Original columns: 223075, Removed: 3767


Report for AliView (strictest):

In [54]:
input_file = "aligned_plastomes_copy_aliview.fasta"

alignment = AlignIO.read(input_file, "fasta")
full_len = 206554

# Example fixed trim region (for report only)
trim_start = 3589  # Replace with real start from AliView
trim_end = 201427  # Replace with real end from AliView

kept_columns = trim_end - trim_start + 1
removed_columns = full_len - kept_columns

print(f"AliView trimmed alignment file: {input_file}")
print(f"Trimmed from columns {trim_start} to {trim_end} ({kept_columns} bp)")
print(f"Original columns: {full_len}, Removed: {removed_columns}")


AliView trimmed alignment file: aligned_plastomes_copy_aliview.fasta
Trimmed from columns 3589 to 201427 (197839 bp)
Original columns: 206554, Removed: 8715


**Step 5:**

Build a plastome phylogeny using IQ-TREE from the trimed and aligned *.fasta* plastomes:
* *trimmed_alignment.fasta*
* *trimmed_alignment_stricter.fasta*
* *aligned_plastomes_copy_aliview.fasta*

In [ ]:
import subprocess
from pathlib import Path

alignment_file = Path(r"C:\Users\terez\OneDrive\Plocha\PP (Plantsies and Programming)\Annonaceae\trimmed_alignment_stricter.fasta")
iqtree_executable = Path(r"D:\iqtree\iqtree-3.0.1-Windows\bin\iqtree3.exe")
output_dir = Path("phylo_output_trimmed_alignment")
output_dir.mkdir(exist_ok=True)

def run_iqtree(alignment_path: Path, threads: int = 4, model: str = "GTR+G"):
    command = [
        str(iqtree_executable),
        "-s", str(alignment_path),
        "-m", model,
        "-nt", str(threads),
        "-bb", "1000",
        "-alrt", "1000",
        "-pre", str(output_dir / "plastome_tree")
    ]

    print(f"Running IQ-TREE with command: {' '.join(command)}")
    try:
        subprocess.run(command, check=True)
    except FileNotFoundError:
        print("❌ IQ-TREE executable not found. Check the path.")
    except subprocess.CalledProcessError as e:
        print(f"❌ IQ-TREE failed with error:\n{e}")

if __name__ == "__main__":
    if alignment_file.exists():
        run_iqtree(alignment_file)
        print("\nTree construction completed. Check output in:", output_dir)
    else:
        print("\nAlignment file not found:", alignment_file)


Running IQ-TREE with command: D:\iqtree\iqtree-3.0.1-Windows\bin\iqtree3.exe -s C:\Users\terez\OneDrive\Plocha\PP (Plantsies and Programming)\Annonaceae\trimmed_alignment_stricter.fasta -m GTR+G -nt 4 -bb 1000 -alrt 1000 -pre phylo_output_trimmed_alignment\plastome_tree


In [ ]:
alignment_file_aliview = Path(r"C:\Users\terez\OneDrive\Plocha\PP (Plantsies and Programming)\Annonaceae\aligned_plastomes_copy_aliview.fasta")
output_dir_aliview = Path("phylo_output_aliview")
output_dir_aliview.mkdir(exist_ok=True)

def run_iqtree_aliview(alignment_path_aliview: Path, threads: int = 4, model: str = "GTR+G"):
    command_aliview = [
        str(iqtree_executable),
        "-s", str(alignment_path_aliview),
        "-m", model,
        "-nt", str(threads),
        "-bb", "1000",
        "-alrt", "1000",
        "-pre", str(output_dir_aliview / "plastome_tree")
    ]

    print(f"Running IQ-TREE with command: {' '.join(command_aliview)}")
    try:
        subprocess.run(command_aliview, check=True)
    except FileNotFoundError:
        print("❌ IQ-TREE executable not found. Check the path.")
    except subprocess.CalledProcessError as e:
        print(f"❌ IQ-TREE failed with error:\n{e}")

if __name__ == "__main__":
    if alignment_file_aliview.exists():
        run_iqtree_aliview(alignment_file_aliview)
        print("\nTree construction completed. Check output in:", output_dir_aliview)
    else:
        print("\nAlignment file not found:", alignment_file_aliview)


Running IQ-TREE with command: D:\iqtree\iqtree-3.0.1-Windows\bin\iqtree3.exe -s C:\Users\terez\OneDrive\Plocha\PP (Plantsies and Programming)\Annonaceae\aligned_plastomes_copy_aliview.fasta -m GTR+G -nt 4 -bb 1000 -alrt 1000 -pre phylo_output_aliview\plastome_tree

Tree construction completed. Check output in: phylo_output_aliview
